[![General Assembly Logo](https://camo.githubusercontent.com/1a91b05b8f4d44b5bbfb83abac2b0996d8e26c92/687474703a2f2f692e696d6775722e636f6d2f6b6538555354712e706e67)](https://generalassemb.ly/education/web-development-immersive)
![Misk Logo](https://i.ibb.co/KmXhJbm/Webp-net-resizeimage-1.png)

*Instructor: Marcus Lim*

# Databases and SQL

## Learning Objectives

### Core

* Connect to a local or remote SQLite database with the command line, Python, and `pandas`
* Perform simple table-level read queries, including:
    * Taking subsets of columns with `SELECT`
    * Taking subsets of rows with `WHERE`, `AND`, and `OR`
    * Aggregating data with `GROUP BY` and `HAVING`
    * Joining tables with `JOIN`
* Eliminate duplicates with `SELECT DISTINCT`
* Perform queries requiring subqueries
* Compare strings with `LIKE`
* Process the output of queries with `AS`, `LIMIT`, and `ORDER BY`

## Introduction

Thus far, you have been performing operations on datasets that can fit on local storage (your computer's hard disk or SSD). However, in a production context, it is far more common for companies to store data on some remote server and extract subsets of it for analysis where necessary. This is for two reasons.

Firstly, the amount of data that a company has might be measurable only in terabytes (thousands of gigabytes). That won't fit on your computer. Also, while `pandas` works great for interactive analysis, it's not built for stability and concurrency. 

This means that data is often stored not in CSVs and JSONs, but rather in *databases*. When you work with a database, you don't need to worry about *how* it stores data, but only about how to access it. At this point, we'll be focusing on *relational databases*, which allow you to treat data like tables of rows and columns, just like you're used to in `pandas`. The strength of these databases is that they make four strong guarantees, which you can remember with **ACID**:

* **A**tomicity (what happens when reads and writes occur simultaneously?)
* **C**onsistency (how do we prevent data corruption?)
* **I**solation (how do we support multiple connections at once?)
* **D**urability (what happens to the data if we lose power?)

The most common way to interact with such databases, by far, is some variant of *Structured Query Language* (SQL). While there exist multiple dialects (sqlite, MySQL, PostgreSQL, Oracle...), the differences are largely minor and syntactical.

## Interacting with databases

We will go through three methods to access and query a database:

* Python using `pandas`
* Python using `sqlite3`
* The command line

Note for Windows users: you may need to download and install the sqlite3 utilities [here](https://sqlite.org/2020/sqlite-tools-win32-x86-3310100.zip).

In [1]:
import sqlite3
import pandas as pd

## SQL with `pandas`

`pandas` can interact with an SQL database using the top-level `read_sql` function. Internally, it uses an `sqlite3` connection object to do so; however, since we are more familiar with `pandas`, we will first look at how we can use `pandas` functionality to perform simple queries, before going specifically into `sqlite3`.

In [2]:
# Create a connection object

conn = sqlite3.connect('titanic_sqlite3')
df = pd.read_sql('SELECT * FROM titanic', conn)

### Taking column subsets

The `SELECT` class is the first piece of SQL we will learn. It takes the following format:

```sql
SELECT <column_names> 
FROM <table_name>
```

The Titanic data has been stored in the `titanic` table. Therefore, for example, if we wanted to get the `survived` and `pclass` columns from it, we would execute `SELECT survived, pclass FROM titanic`:

In [3]:
sql = '''
SELECT survived, pclass
FROM titanic
'''

result = pd.read_sql(sql, conn)
result.head()

,survived,pclass
0,0,3
1,1,1
2,1,3
3,1,1
4,0,3


What if we want all the columns? In this case, SQL provides a special wildcard, `*`. It's okay to use a bare `SELECT *` in this case because we have very few rows, but note that doing this on a database with, say, a hundred million rows will probably just lead to a timeout.

In [4]:
sql = '''
SELECT * 
FROM titanic'''

result = pd.read_sql(sql, conn)
result.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


We can say in general that a `SELECT` class is equivalent to `df[column_names]`, where `column_names` is a `list` of column names in `df`.

### Unique values

We can modify the `SELECT` clause to `SELECT DISTINCT` to get only the unique values in a column, or the unique combinations of values in a number of columns. Above, we got the values in the `survived` and `pclass` columns for all passengers. With `SELECT DISTINCT`, we can see what pairs exist.

In [5]:
sql = '''
SELECT DISTINCT survived, pclass
FROM titanic
'''

pd.read_sql(sql, conn)

,survived,pclass
0,0,3
1,1,1
2,1,3
3,0,1
4,1,2
5,0,2


### Taking row subsets

Sometimes, as in `pandas`, we want to filter rows based on the values of certain columns, which is done with a `WHERE` clause, which has a format as follows: 

```sql
SELECT <column_names> 
FROM <table_name> 
WHERE <condition>
```

The condition can be expressed as it is in normal Python, with operators such as `!=` and `<`. Note, however, that the equality operator is `=`, not `==`.

Say we want all columns where `survived` is 1:

In [6]:
sql = '''
SELECT *
FROM titanic
WHERE survived = 1
'''

result = pd.read_sql(sql, conn)
result['survived'].unique()

array([1])

As in `pandas`, we may also want to combine conditions, which we can do with `AND` and `OR`. We can add on an additional filter, selecting only those rows where `age` is less than 20 or more than 50:

In [7]:
sql = '''
SELECT *
FROM titanic
WHERE survived = 1
AND (age < 20 OR age > 50)
'''

result = pd.read_sql(sql, conn)
result['age'].between(20, 50).any()

False

Think about how you would perform these queries in `pandas`.

### String comparisons

Apart from the standard operators you know, SQL has a `LIKE` clause, which allows you to specify a sort of pattern for strings, with two wildcards:

* `%`, which represents any number of (possibly different) characters
* `_`, which represents any single character

So, for example, to get all the rows where the `name` column contains `Mr.`:

```sql
SELECT *
FROM titanic
WHERE name LIKE '%Mr.%'
```

The reverse operation (match those strings which do *not* conform to this pattern) can be performed with `NOT LIKE`.


In [8]:
sql = '''
SELECT *
FROM titanic
WHERE name LIKE '%Mr.%'
'''

result = pd.read_sql(sql, conn)
result.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S
2,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,None,Q
3,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
4,13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,None,S


### In-class exercise

Let's practice some SQL! For the questions below, answer them first using solely `pandas`, and then write a SQL query and execute it with `pd.read_sql`. Check your answers against the `pandas` result!

**1. Select the `name`, `sex` and `age` columns.**

In [18]:
df[['name', 'sex', 'age']]

,name,sex,age
0,"Braund, Mr. Owen Harris",male,22.0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0
2,"Heikkinen, Miss. Laina",female,26.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0
4,"Allen, Mr. William Henry",male,35.0
...,...,...,...
886,"Montvila, Rev. Juozas",male,27.0
887,"Graham, Miss. Margaret Edith",female,19.0
888,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN
889,"Behr, Mr. Karl Howell",male,26.0


In [20]:
sql = '''
SELECT name, sex, age
FROM titanic
'''

result = pd.read_sql(sql, conn)
result.head()

,name,sex,age
0,"Braund, Mr. Owen Harris",male,22.0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0
2,"Heikkinen, Miss. Laina",female,26.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0
4,"Allen, Mr. William Henry",male,35.0


**2. Get all the rows where `fare` is less than 50.**

In [17]:
sql = '''
SELECT *
FROM titanic
WHERE fare < 50 
'''

result = pd.read_sql(sql, conn)
result.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
2,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S
3,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,None,Q
4,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,None,S


**3. Get the unique combinations of the `survived`, `pclass` and `embarked` columns.**

In [11]:
sql = '''
SELECT DISTINCT survived, pclass, embarked
FROM titanic
'''

result = pd.read_sql(sql, conn)
result.head()

,survived,pclass,embarked
0,0,3,S
1,1,1,C
2,1,3,S
3,1,1,S
4,0,3,Q


**4. Get the ages of all the male passengers whose Passenger ID is more than 400.**

In [14]:
sql = '''
SELECT *
FROM titanic
WHERE passengerid > 400
AND (sex = 'male')
'''

result = pd.read_sql(sql, conn)
result.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,401,1,3,"Niskanen, Mr. Juha",male,39.0,0,0,STON/O 2. 3101289,7.925,None,S
1,402,0,3,"Adams, Mr. John",male,26.0,0,0,341826,8.050,None,S
2,404,0,3,"Hakkarainen, Mr. Pekka Pietari",male,28.0,1,0,STON/O2. 3101279,15.850,None,S
3,406,0,2,"Gale, Mr. Shadrach",male,34.0,1,0,28664,21.000,None,S
4,407,0,3,"Widegren, Mr. Carl/Charles Peter",male,51.0,0,0,347064,7.750,None,S


**5. Get the names of the female passengers whose names do not contain "Miss.".**

In [16]:
sql = '''
SELECT *
FROM titanic
WHERE sex = 'female'
AND name NOT LIKE '%Miss%'
'''

result = pd.read_sql(sql, conn)
result.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
2,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
3,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,None,S
4,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,None,C


### Sorting

The result of a query can be sorted with the `ORDER BY` clause, which has the following format:

```sql
SELECT ...
ORDER BY <order_column_names>
```

You may optionally specify ascending order with `ASC` (the default) and descending order with `DESC` on a column-by-column basis. So, for example, if you wanted to order by the `sibsp` column in ascending order and then the `parch` column in descending order, you could execute the following query:

```sql
SELECT *
FROM titanic
ORDER BY sibsp ASC,
         parch DESC
```


In [21]:
sql = '''
SELECT *
FROM titanic
ORDER BY sibsp ASC,
         parch DESC
'''

result = pd.read_sql(sql, conn)
result.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,639,0,3,"Panula, Mrs. Juha (Maria Emilia Ojala)",female,41.0,0,5,3101295,39.6875,None,S
1,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,None,Q
2,568,0,3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",female,29.0,0,4,349909,21.0750,None,S
3,859,1,3,"Baclini, Mrs. Solomon (Latifa Qurban)",female,24.0,0,3,2666,19.2583,None,C
4,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,None,S


### Limiting results

In some cases, you only want a subset of the returned results; for example, the first three. This is achieved with the `LIMIT` clause, which has this format:

```sql
SELECT ...
LIMIT <result_count>
```

If the number of results returned is lower than the specified result count, then the `LIMIT` clause has no effect.

In [22]:
sql = '''
SELECT *
FROM titanic
LIMIT 7
'''

pd.read_sql(sql, conn)

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,None,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S


### Aggregating

You may also wish to perform aggregation operations, which can be done with function-like clauses. The ones you should know:

* `COUNT`
* `AVG`
* `SUM`
* `MIN`
* `MAX`

They operate the same way as in `pandas` - `AVG` is the equivalent of `mean`. Note that `count` counts only *non-null* values!

To use them, just surround column names with the desired aggregation function. For example, say we wanted, for all passengers, the following data:

* The number of non-null values in the `cabin` column
* The percentage of people who survived
* The total fare paid
* The maximum age
* The minimum age

In [23]:
sql = '''
SELECT COUNT(cabin), 
       AVG(survived), 
       SUM(fare), 
       MAX(age), 
       MIN(age)
FROM titanic
'''

pd.read_sql(sql, conn)

,COUNT(cabin),AVG(survived),SUM(fare),MAX(age),MIN(age)
0,204,0.383838,28693.9493,80.0,0.42


This looks almost right, except for the fact that the survival percentage is on the wrong scale (it needs to be multiplied by 100 to be a percentage). It turns out that you can also perform arithmetic operations in SQL:

In [24]:
sql = '''
SELECT COUNT(cabin), 
       AVG(survived) * 100, 
       SUM(fare), 
       MAX(age), 
       MIN(age)
FROM titanic
'''

pd.read_sql(sql, conn)

,COUNT(cabin),AVG(survived) * 100,SUM(fare),MAX(age),MIN(age)
0,204,38.383838,28693.9493,80.0,0.42


Lastly, it would help if we could rename the results to be more descriptive. This can be done with the `AS` clause:

In [25]:
sql = '''
SELECT COUNT(cabin) AS cabin_count, 
       AVG(survived) * 100 AS survival_percentage, 
       SUM(fare) AS total_fare, 
       MAX(age) AS max_age, 
       MIN(age) AS min_age
FROM titanic
'''

pd.read_sql(sql, conn)

,cabin_count,survival_percentage,total_fare,max_age,min_age
0,204,38.383838,28693.9493,80.0,0.42


### Groupby aggregations

This aggregation functionality is incomplete without the power of groupby operations, and in fact the `pandas` groupby functionality is inspired by SQL. The relevant clause is, of course, `GROUP BY`, which takes the following format:

```sql
SELECT <column_names> 
FROM <table_name> 
GROUP BY <groupby_column_names>
```

In `pandas`, if you wanted to get the max age by `pclass`, you would execute the following code:

```python
df.groupby('pclass')['age'].max()
```

The SQL equivalent:

```sql
SELECT MAX(age) 
FROM titanic 
GROUP BY pclass
```

In [26]:
sql = '''
SELECT MAX(age) 
FROM titanic 
GROUP BY pclass
'''

pd.read_sql(sql, conn)

,MAX(age)
0,80.0
1,70.0
2,74.0


Note that unlike in `pandas`, the grouping column is not automatically included for you; if you want to know which value corresponds to which aggregation, you need to include it explicitly:

In [27]:
sql = '''
SELECT pclass, MAX(age) 
FROM titanic 
GROUP BY pclass
'''

pd.read_sql(sql, conn)

,pclass,MAX(age)
0,1,80.0
1,2,70.0
2,3,74.0


Grouping on multiple columns is similarly supported:

In [28]:
sql = '''
SELECT survived, pclass, MAX(age) 
FROM titanic 
GROUP BY survived, pclass
'''

result = pd.read_sql(sql, conn)
result

,survived,pclass,MAX(age)
0,0,1,71.0
1,0,2,70.0
2,0,3,74.0
3,1,1,80.0
4,1,2,62.0
5,1,3,63.0


While it is not necessary, you can combine a `WHERE` clause with a `GROUP BY` clause so that you groupby only after filtering:

In [29]:
sql = '''
SELECT survived, pclass, MAX(age) 
FROM titanic 
WHERE sex = 'male'
GROUP BY survived, pclass
'''

pd.read_sql(sql, conn)

,survived,pclass,MAX(age)
0,0,1,71.0
1,0,2,70.0
2,0,3,74.0
3,1,1,80.0
4,1,2,62.0
5,1,3,45.0


This is equivalent to the following:

```python
df[df['sex'] == 'male'].groupby(['survived', 'pclass']).max()
```

### In-class exercise 2

More SQL practice. You know how it works.

**1. Get the highest value of the `sibsp` column.**

In [30]:
sql = '''
SELECT MAX(sibsp)
FROM titanic
'''

result = pd.read_sql(sql, conn)
result

,MAX(sibsp)
0,8


**2. Get the ages of the 5 oldest passengers.**

In [56]:
sql = '''
select age 
from titanic
order by 1 desc
limit 5
'''

result = pd.read_sql(sql, conn)
result

,age
0,80.0
1,74.0
2,71.0
3,71.0
4,70.5


**3. Get the counts of passengers at each port of embarkation (that's the `embarked` column).**

In [33]:
sql = '''
SELECT embarked, COUNT(passengerid) AS numbe_of_passengers
FROM titanic 
GROUP BY embarked
'''

result = pd.read_sql(sql, conn)
result

,embarked,numbe_of_passengers
0,None,2
1,C,168
2,Q,77
3,S,644


**4. Get the percentage of passengers who survived, divided by passenger class, and then sex. Rename the columns in the result appropriately.**

In [37]:
sql = '''
SELECT sex, pclass, AVG(survived) * 100 AS survived_passenger_perc
FROM titanic
GROUP BY sex, pclass
'''

result = pd.read_sql(sql, conn)
result

,sex,pclass,survived_passenger_perc
0,female,1,96.808511
1,female,2,92.105263
2,female,3,50.000000
3,male,1,36.885246
4,male,2,15.740741
5,male,3,13.544669


**5. Get the number of passengers in C-type cabins.**

In [54]:
sql = '''
SELECT pclass, COUNT(passengerid)
FROM titanic
WHERE pclass = '3'
'''

result = pd.read_sql(sql, conn)
result

,pclass,COUNT(passengerid)
0,3,491


### Groupby conditions

You can also impose conditions on the groups that are created by a `GROUP BY` clause. This is done not with `WHERE`, which is for filtering *before* the grouping process, but with `HAVING`:

```sql
SELECT <column_names>
FROM <table_name>
GROUP BY <groupby_column_name>
```

We can use this to, for example, get the values of `embarked` where the mean age is more than 30.

In [ ]:
sql = '''
SELECT embarked
FROM titanic
GROUP BY embarked
HAVING AVG(age) > 30
'''

result = pd.read_sql(sql, conn)
result.head()

This is equivalent to `df.groupby('embarked').filter(lambda g: g['age'].mean() > 30)['embarked'].unique()`, except that SQL also includes the null group.

### Joins

Joins can be performed in SQL with the `JOIN` and `ON` clauses, in the following format:

```sql
SELECT <column_names>
FROM <left_table_name>
<JOIN_TYPE> JOIN <right_table_name>
ON <left_table_name>.<left_table_join_column> = <right_table_name>.<right_table_join_column>
```

So, for example, say you want to join the `titanic` table (left) and the `classes` table (right):

In [ ]:
sql = '''
SELECT *
FROM titanic
LEFT JOIN classes
ON titanic.pclass = classes.pclass
'''

result = pd.read_sql(sql, conn)
result.head()

The result has two `pclass` columns, one from each table. Perform the same query, but this time keeping only the `passengerid` and `pclass_name` columns.

In [ ]:
sql = '''
SELECT passengerid, pclass_name
FROM titanic
LEFT JOIN classes
ON titanic.pclass = classes.pclass
'''

result = pd.read_sql(sql, conn)
result.head()

Some rows have `None` as a value for `pclass_name`. To understand why, examine the `classes` table:

In [ ]:
sql = '''
SELECT *
FROM classes
'''

pd.read_sql(sql, conn)

Feel free to experiment on your own with other join types.

### Subqueries

Sometimes, you need to perform queries within queries. With what we have learnt above, it is impossible to get, for example, the IDs of all passengers who are older than average. In `pandas`, we would do it in this way:

```python
df.loc[df['age'] > df['age'].mean(), 'passengerid']
```

Notice that we need to compare the `age` column to another value that is itself derived from the `age` column. In such a case, we need a *subquery*:

```sql
SELECT passengerid
FROM titanic
WHERE age > (
    SELECT AVG(age)
    FROM titanic
)
AND age IS NOT NULL
```

In this subquery, we first get the average age from a query. We then *nest*, or insert, the result of that query into the larger query that makes a comparison on the `age` column, while selecting only the `passengerid` column.

In [ ]:
sql = '''
SELECT passengerid
FROM titanic
WHERE age > (
    SELECT AVG(age)
    FROM titanic
)
AND age IS NOT NULL
'''

result = pd.read_sql(sql, conn)
result.head()

## SQL with `sqlite3`

The `sqlite3` library allows you to interact with a SQLite database at a lower level. `pandas` takes an SQL query and a connection object, hiding some parts of the hard work from us. Now, let's try to use `sqlite` directly. First, we need a *cursor object*:

In [ ]:
cursor = conn.cursor()

We can execute SQL queries by calling `.execute()` on the cursor, which represents the set of results that are returned from a query. Notice that it is the `fetchall` call that actually interfaces with the database to retrieve data:

In [ ]:
%%timeit

cursor.execute('SELECT * FROM titanic')

In [ ]:
%%timeit

cursor.execute('SELECT * FROM titanic')
cursor.fetchall()

The queries we have learnt above deal with *reading* data. However, there may be cases in which you need to perform *insertions* or *modifications*, which we will talk about in the next lesson.

### SQL with the command line

You can access the database in this repo with `sqlite3 titanic_sqlite3`. All the above operations can be performed through the command line.

Each dialect of SQL has its own client for accessing a database. For example, PostgreSQL uses `psql`. Consult the documentation of your client.